* Think of a word or words that define each point  
    * Proximity to each of those words creates a new centroid.
    * proximity is determined using pre-trained word2vec
    * basically, check how close a new word is to each of the centroids (which are defined by words)
    * how close it is to each centroid contributes contributes to it's distance in that direction

In [1]:
import pandas as pd

import pickle

from scipy.spatial import minkowski_distance

Find songs closest to each centroid and define those moods

In [2]:
pickle_in = open('../Data/kmeans_cluster_centers_11', 'rb')
centroids = pickle.load(pickle_in)
pickle_in.close()

In [3]:
df = pd.read_pickle('../Data/labeled_points')

In [4]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

In [5]:
features = df.select_dtypes(include=numerics)
no_features = df.select_dtypes(exclude=numerics)

In [6]:
centroids[0]

array([-0.46083014,  0.16916763,  0.40099883, -0.13042313, -0.06470108,
        0.38606049, -0.06526977,  0.2656626 , -0.33491291, -1.30954252,
        1.71504212,  0.83495209,  1.06035826,  0.601207  ])

In [7]:
def match_sentiment(sentiment_df, feats_to_match):
    distances_df = sentiment_df.apply(lambda x:
                                      minkowski_distance(x.values,
                                                         feats_to_match, p=.5),
                                      axis=1)
    distances_df = pd.DataFrame(distances_df)
    distances_df.columns = ['distance']
    n_smallest = distances_df.nsmallest(15, 'distance').index
    return n_smallest

In [8]:
n_smallest = match_sentiment(features.drop('labels', axis=1), centroids[1])

In [9]:
song_info = pd.read_pickle('../Data/combined_genius_spotify_uncleaned')

In [10]:
song_info.drop_duplicates(subset='spotify_song_id', inplace=True)

In [11]:
df.head()

,album_uri,analysis_url,artist_uri,spotify_song_id,track_href,type,uri,acousticness,danceability,energy,...,loudness,speechiness,valence,neg,neu,pos,compound,polarity,subjectivity,labels
0,spotify:album:4vQcWE7Ce6ORsatrqXkBtt,https://api.spotify.com/v1/audio-analysis/2dqp...,spotify:artist:14H7ag1wpQOsPPQJOD6Dqr,2dqpsVfEE3SWltmZ7UWb0m,https://api.spotify.com/v1/tracks/2dqpsVfEE3SW...,audio_features,spotify:track:2dqpsVfEE3SWltmZ7UWb0m,-0.889816,-0.150869,0.237184,...,0.146473,-0.392443,1.159519,-1.319268,1.358555,-0.450702,0.564429,-0.768552,-1.536352,3
1,spotify:album:4vQcWE7Ce6ORsatrqXkBtt,https://api.spotify.com/v1/audio-analysis/3C6f...,spotify:artist:14H7ag1wpQOsPPQJOD6Dqr,3C6fiZI9I8xEMbUSfXfQCD,https://api.spotify.com/v1/tracks/3C6fiZI9I8xE...,audio_features,spotify:track:3C6fiZI9I8xEMbUSfXfQCD,0.694877,-1.178074,-0.849247,...,-0.004912,-0.434665,-0.865222,-1.319268,0.284414,0.737061,0.749197,-0.436594,0.166627,5
2,spotify:album:4vQcWE7Ce6ORsatrqXkBtt,https://api.spotify.com/v1/audio-analysis/0gBT...,spotify:artist:14H7ag1wpQOsPPQJOD6Dqr,0gBTPgvzYPMPbYlTuPjx9n,https://api.spotify.com/v1/tracks/0gBTPgvzYPMP...,audio_features,spotify:track:0gBTPgvzYPMPbYlTuPjx9n,1.500189,0.349394,-1.017385,...,-0.813355,-0.478806,-0.236572,0.085411,1.276455,-1.479591,-1.684092,-0.300256,0.242617,4
3,spotify:album:4vQcWE7Ce6ORsatrqXkBtt,https://api.spotify.com/v1/audio-analysis/0QvF...,spotify:artist:14H7ag1wpQOsPPQJOD6Dqr,0QvF5Q3dg3TpRsU6Wqz9EV,https://api.spotify.com/v1/tracks/0QvF5Q3dg3Tp...,audio_features,spotify:track:0QvF5Q3dg3TpRsU6Wqz9EV,-0.275229,-1.291467,-0.590573,...,0.061666,-0.298402,-1.651850,-0.493545,-1.159177,1.675167,0.880013,1.111310,-0.044095,1
4,spotify:album:4vQcWE7Ce6ORsatrqXkBtt,https://api.spotify.com/v1/audio-analysis/3G5q...,spotify:artist:14H7ag1wpQOsPPQJOD6Dqr,3G5qMSP9AywIErX4HSbP5a,https://api.spotify.com/v1/tracks/3G5qMSP9AywI...,audio_features,spotify:track:3G5qMSP9AywIErX4HSbP5a,-0.576832,0.622871,-0.366389,...,0.085444,-0.478806,-0.608047,1.528054,-0.693944,-0.443136,-1.684092,-0.123401,1.329544,9


In [12]:
df = df.merge(song_info, on='spotify_song_id')

In [17]:
for i in n_smallest:
    print(df.iloc[i].uri,
          df.iloc[i].spotify_artist_name,
          df.iloc[i].song_title,
          '\n')

spotify:track:3yaHtRv2MumopCUS6tspPp Meiko deep sweat 

spotify:track:2ZRvcRcKdwQBjTAqj2XNJN Grizzly Bear losing all sense 

spotify:track:1u6sC8u9G745oIX7GGjSNM Sambassadeur days 

spotify:track:20yahLUVOWqoJsRjZtBQFA Generationals black and white 

spotify:track:1QkeR4Hz062bENV5DBSdyx The Magnetic Fields swinging london 

spotify:track:1zqQz3SYr0f3HPGetu1a9E The Go-Betweens spirit of a vampyre 

spotify:track:4fVN9KMh432ZbxYoqtSlOr The Whitlams stay with me 

spotify:track:6zUJkJyfFOOKmujVQ3wXDe Jens Lekman the end of the world is bigger than love 

spotify:track:0ekXXkpMdvnP2FyCvW4oAp Vinyl Theatre new machines 

spotify:track:7p1qh5U28egQmv0umbM5Ld Fitz and The Tantrums news u 

spotify:track:19yhNQkm3HPS2DUBzzn7VY Fanfarlo a distance 

spotify:track:6KG5ooGkPbDQqCseW73LRl Gold Motel counter clockwise 

spotify:track:4OPsp7k2L49q2SUpPpb8go Sondre Lerche two way monologue 

spotify:track:2OUI5IpovB2j1hlP0lb5Jv Yeasayer sunrise 

spotify:track:1qGAxJaUOr2ro8HaDI1GXx The Whitlams tang